In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!mkdir ~/.kaggle

In [ ]:
!cp /content/drive/MyDrive/KAGGLE_API_CREDENTIALS/kaggle.json ~/.kaggle/kaggle.json

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
import torch
from torch.optim import Adam
import matplotlib.pyplot as plt
from torch import nn
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report
import copy

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
!kaggle datasets download -d mdwaquarazam/agricultural-crops-image-classification

In [ ]:
!unzip /content/agricultural-crops-image-classification.zip

In [ ]:
import numpy as np
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
import copy

# Define the transformations
train_transform = transforms.Compose([
    transforms.RandomRotation(10),
    transforms.Resize((180,180)),
    transforms.ToTensor(),
])

test_transform = transforms.Compose([
    transforms.Resize((180,180)),
    transforms.ToTensor()
])

# Load the datasets with ImageFolder
dataset = datasets.ImageFolder('/content/Agricultural-crops')

# Get the indices for the split
random_seed = 42
test_split = 0.33
dataset_size = len(dataset)
indices = list(range(dataset_size))
split = int(np.floor(test_split * dataset_size))

np.random.seed(random_seed)
np.random.shuffle(indices)

train_indices, test_indices = indices[split:], indices[:split]

# Create SubsetRandomSamplers
train_sampler = SubsetRandomSampler(train_indices)
test_sampler = SubsetRandomSampler(test_indices)

# Create the DataLoaders
#the deep copy is used to ensure that the transformations applied to the training dataset don't affect the testing dataset and vice versa.
train_loader = DataLoader(copy.deepcopy(dataset), batch_size=32, sampler=train_sampler)
test_loader = DataLoader(copy.deepcopy(dataset), batch_size=32, sampler=test_sampler)

# Now apply the transformations on the datasets
train_loader.dataset.transform = train_transform
test_loader.dataset.transform = test_transform


In [ ]:

class CROP_CLASSIFICATION_Model_V1(nn.Module):
  def __init__(self, input_shape , output_shape):
    super().__init__()

    self.conv_block_1 = nn.Sequential(
        nn.Conv2d(in_channels=input_shape, out_channels=96, stride=1, kernel_size=3, padding=1),
        nn.ReLU(),
        nn.Conv2d(in_channels = 96, out_channels=108, stride=1, kernel_size=3, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2)
    )

    self.conv_block_2 = nn.Sequential(
        nn.Conv2d(in_channels=108, out_channels=120, stride=1, kernel_size=3, padding=1),
        nn.ReLU(),
        nn.Conv2d(in_channels = 120, out_channels=140, stride=1, kernel_size=3, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2)
    )

    self.conv_block_3 = nn.Sequential(
        nn.Conv2d(in_channels=140, out_channels=160, stride=1, kernel_size=3, padding=1),
        nn.ReLU(),
        nn.Conv2d(in_channels = 160, out_channels=200, stride=1, kernel_size=3, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2)
    )

    self.conv_block_4 = nn.Sequential(
        nn.Conv2d(in_channels=200, out_channels=260, stride=1, kernel_size=3, padding=1),
        nn.ReLU(),
        nn.Conv2d(in_channels = 260, out_channels=300, stride=1, kernel_size=3, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2)
    )

    self.classifier_layer = nn.Sequential(
        nn.Flatten(),
        nn.Linear(in_features=36300,out_features=output_shape)
    )

  def forward(self,X):
    X = self.conv_block_1(X)
    X = self.conv_block_2(X)
    X = self.conv_block_3(X)
    X = self.conv_block_4(X)
    X = self.classifier_layer(X)

    return X


In [ ]:

class CROP_CLASSIFICATION_Model_V2(nn.Module):
  def __init__(self, input_shape , output_shape):
    super().__init__()

    self.conv_block_1 = nn.Sequential(
        nn.Conv2d(in_channels=input_shape, out_channels=96, stride=1, kernel_size=5),
        nn.ReLU(),
        nn.Conv2d(in_channels=96, out_channels=108, stride=1, kernel_size=5),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2)
    )

    self.conv_block_2 = nn.Sequential(
        nn.Conv2d(in_channels=108, out_channels=200, stride=1, kernel_size=5),
        nn.ReLU(),
        nn.Conv2d(in_channels=200, out_channels=260, stride=1, kernel_size=5),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2)
    )

    self.conv_block_3 = nn.Sequential(
        nn.Conv2d(in_channels=260, out_channels=300, stride=1, kernel_size=3),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2)
    )

    self.classifier_layer = nn.Sequential(
        nn.Flatten(),
        nn.Linear(in_features=97200, out_features=output_shape)
    )

  def forward(self, X):
    X = self.conv_block_1(X)
    X = self.conv_block_2(X)
    X = self.conv_block_3(X)
    X = self.classifier_layer(X)

    return X


In [ ]:
#model_0 = CROP_CLASSIFICATION_Model_V1(input_shape=3,output_shape=len(test_loader.dataset.classes)).to(device)
model_1 = CROP_CLASSIFICATION_Model_V2(input_shape=3,output_shape=len(test_loader.dataset.classes)).to(device)

In [ ]:
from collections import Counter

# Get a list of all the labels in the dataset
all_labels = [label for _, label in dataset]

# Calculate the number of samples per class
counts = Counter(all_labels)
# Calculate the total number of samples
total_count = sum(counts.values())

# Calculate weights as the inverse of the class frequencies
weights = {class_id: total_count/count for class_id, count in counts.items()}

# Now you have a dictionary of class weights, but we need a tensor of weights for the loss function.
# Also, the weight tensor should have the same order of classes as in the dataset.
weights_tensor = torch.tensor([weights[i] for i in range(len(weights))])


In [ ]:
weights_tensor = weights_tensor.to(device)
loss_fn = nn.CrossEntropyLoss(weight=weights_tensor)

In [ ]:

def train_stage(model=None,loss_fn=None, learning_rate=None,train_loader=None):


  optimizer = Adam(params=model.parameters(), lr=learning_rate)
  model.train()

  for X_train,y_train in train_loader:
    X_train,y_train = X_train.to(device), y_train.to(device)
    optimizer.zero_grad()
    y_pred = model(X_train)
    train_loss = loss_fn(y_pred,y_train)
    train_loss.backward()
    optimizer.step()


In [ ]:
def test_stage(model=None,test_loader=None):
  with torch.inference_mode():
    model.eval()
    y_trues = []
    y_preds = []
    for X_test,y_test in test_loader:
      X_test, y_test = X_test.to(device), y_test.to(device)
      y_pred = model(X_test)

      y_trues.extend(list(y_test.cpu().numpy()))
      y_preds.extend(list(y_pred.argmax(dim=1).cpu().numpy()))

    return (y_trues,y_preds)

In [ ]:
epochs = 100
best_accuracy = 0.0
best_report = None
best_model_weights = model_1.state_dict()
from sklearn.metrics import classification_report

for epoch in range(epochs):
  train_stage(model=model_1,loss_fn=loss_fn,learning_rate=0.01,train_loader=train_loader)

  if epoch % 5 == 0:
    y_trues, y_preds = test_stage(model=model_1, test_loader=test_loader)
    report = classification_report(
        y_true=y_trues,
        y_pred=y_preds,
        output_dict=True,
        target_names=test_loader.dataset.classes,
        zero_division='warn'
    )

    if report['accuracy'] > best_accuracy:
      best_report = report
      best_model_weights = copy.deepcopy(model_1.state_dict())

  print(epoch)



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0
1
2
3
4


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


5
6
7
8
9


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


...

85
86
87
88
89


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


90
91
92
93
94


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


95
96
97
98
99


In [ ]:
print(best_report)